In [12]:
import numpy as np
import cv2 
import numpy as np
import import_ipynb
import operator
from utils import distance_between


In [6]:
def grayscale(img):
    NUM_CHANNELS = 3
    if len(img.shape) == NUM_CHANNELS:
        grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return grayscale

In [7]:
def convert_to_binary(img):
    MAX_THRESHOLD_VALUE = 255
    BLOCK_SIZE = 7
    THRESHOLD_CONSTANT = 0
    
    grayscale = cv2.GaussianBlur(img, (7, 7), 0)
    filtered = cv2.adaptiveThreshold(~grayscale, MAX_THRESHOLD_VALUE, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, BLOCK_SIZE, THRESHOLD_CONSTANT)
    
    return filtered

In [8]:
def perspective_crop(img):
    
#     find external contours
    contours, h = cv2.findContours(img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    
#     Sort by area, descending

    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    polygon = contours[0]  # Largest image
    
#     Find extreme points

    bottom_right, _ = max(enumerate([pt[0][0] + pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
    top_left, _ = min(enumerate([pt[0][0] + pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
    bottom_left, _ = min(enumerate([pt[0][0] - pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
    top_right, _ = max(enumerate([pt[0][0] - pt[0][1] for pt in polygon]), key=operator.itemgetter(1))


    crop_rect = polygon[top_left][0], polygon[top_right][0], polygon[bottom_right][0], polygon[bottom_left][0]

    top_left, top_right, bottom_right, bottom_left = crop_rect[0], crop_rect[1], crop_rect[2], crop_rect[3]

    # Explicitly set the data type to float32 or `getPerspectiveTransform` will throw an error
    src = np.array([top_left, top_right, bottom_right, bottom_left], dtype='float32')

    # Get the longest side in the rectangle
    side = max([
        distance_between(bottom_right, top_right),
        distance_between(top_left, bottom_left),
        distance_between(bottom_right, bottom_left),
        distance_between(top_left, top_right)
    ])

    dst = np.array([[0, 0], [side - 1, 0], [side - 1, side - 1], [0, side - 1]], dtype='float32')

    m = cv2.getPerspectiveTransform(src, dst)
    crop = cv2.warpPerspective(img, m, (int(side), int(side)))
    
    return crop


In [9]:
def remove_noise(img):
    kernel = np.ones((2,2),np.uint8)
    img = cv2.erode(img, kernel)
    img = cv2.dilate(img, kernel)
    img = cv2.medianBlur(img,5)
    
    return img

In [13]:
def svm(img):
    img_resized = cv2.resize(img, (32,32))
    return img_resized.flatten()

In [14]:
def cnn(img):
    img_resized = cv2.resize(img, (64,64))
    return np.reshape(img_resized, (64, 64, 1))/255.0